In [1]:
import json
from pathlib import Path
from openai._client import OpenAI

In [3]:
with open("../apikeys/apikeys.json") as f:
    api_keys = json.load(f)

### TTS

In [26]:
def read_and_prepare_text(filepath, max_len=4096):
    parts = []
    buffer = []

    def buffer_size():
        return sum([len(i) for i in buffer])

    with open(filepath) as f:
        for line in f:
            line = line.strip()
            if buffer_size() + len(line) >= max_len:
                parts.append("\n".join(buffer))
                buffer = []
            buffer.append(line)

    if buffer:
        parts.append("\n".join(buffer))
    return parts

In [31]:
filepath = "ch2.txt"
text = read_and_prepare_text(filepath, max_len=1024)

In [32]:
print(len(text))

30


In [33]:
client = OpenAI(
    base_url='https://oneapi.xty.app/v1',
    api_key=api_keys["3.5"]
)

In [34]:
revised_text = []

In [35]:
for i, t in enumerate(text):
    print(f"processing text part {i}..")
    response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[
            {"role": "system", "content": "我会发给你一段文字，请将文字中出现的错别字进行改正、对出现的漏字现象进行补全、删除多余的空格。你不需要告诉我错在哪里，也不需要告诉我如何改正，你直接把改好后的文字发出来就可以了。"},
            {"role": "user", "content": t},
        ]
    )
    resp_message = response.choices[0].message.dict()
    revised_text.append(resp_message["content"])

processing text part 0..
processing text part 1..
processing text part 2..
processing text part 3..
processing text part 4..
processing text part 5..
processing text part 6..
processing text part 7..
processing text part 8..
processing text part 9..
processing text part 10..
processing text part 11..
processing text part 12..
processing text part 13..
processing text part 14..
processing text part 15..
processing text part 16..
processing text part 17..
processing text part 18..
processing text part 19..
processing text part 20..
processing text part 21..
processing text part 22..
processing text part 23..
processing text part 24..
processing text part 25..
processing text part 26..
processing text part 27..
processing text part 28..
processing text part 29..


In [36]:
len(revised_text)

30

In [39]:
for section_no, content in enumerate(revised_text):
    if section_no < 13:
        continue
    print(f"tts for {section_no} (length: {len(content)})")
    response = tts_client.audio.speech.create(
        model="tts-1",
        voice="nova",
        input=content[:1000]
    )
    response.stream_to_file(f"影响力_ch2_part_{section_no}.mp3")


tts for 13 (length: 747)
tts for 14 (length: 781)
tts for 15 (length: 858)
tts for 16 (length: 933)
tts for 17 (length: 991)
tts for 18 (length: 750)
tts for 19 (length: 477)
tts for 20 (length: 942)
tts for 21 (length: 975)
tts for 22 (length: 1009)
tts for 23 (length: 996)
tts for 24 (length: 876)
tts for 25 (length: 823)
tts for 26 (length: 805)
tts for 27 (length: 930)
tts for 28 (length: 898)
tts for 29 (length: 128)


In [40]:
!zip -r 影响力_ch2.zip 影响力_ch2_part_*

zsh:1: command not found: zip
